In [1]:
!nvidia-smi

Thu Dec  2 10:02:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    42W / 350W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
from datasets import load_dataset, load_metric
from datasets import Dataset

In [3]:
import pandas as pd

In [4]:
train_rank_dataset_tot_pd = pd.read_json('data_david/ranked_temporal_pt1.json')

In [5]:
val_rank_dataset_tot_pd = pd.read_csv('data_david/ranked_temporal_val_df.csv', engine='python', encoding='utf8', error_bad_lines=False, header=0, names=['text','target'])

In [6]:
train_rank_dataset_tot_pd.shape

(60000, 2)

In [7]:
for i in range(train_rank_dataset_tot_pd.shape[0]):
    if isinstance(train_rank_dataset_tot_pd['text'].get(i),str) == False:
        train_rank_dataset_tot_pd.drop(i,inplace=True)

In [8]:
for i in range(val_rank_dataset_tot_pd.shape[0]):
    if isinstance(val_rank_dataset_tot_pd['text'].get(i),str) == False:
        val_rank_dataset_tot_pd.drop(i,inplace=True)

In [9]:
train_rank_dataset_tot = Dataset.from_pandas(train_rank_dataset_tot_pd)
val_rank_dataset_tot = Dataset.from_pandas(val_rank_dataset_tot_pd)

In [10]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=4):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [11]:
from datasets import load_dataset, load_metric
from transformers import LongformerTokenizer, EncoderDecoderModel

model = EncoderDecoderModel.from_pretrained("patrickvonplaten/longformer2roberta-cnn_dailymail-fp16")
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
model.to('cuda')

Some weights of the model checkpoint at patrickvonplaten/longformer2roberta-cnn_dailymail-fp16 were not used when initializing EncoderDecoderModel: ['decoder.roberta.pooler.dense.weight', 'decoder.roberta.pooler.dense.bias']
- This IS expected if you are initializing EncoderDecoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EncoderDecoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


EncoderDecoderModel(
  (encoder): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0): LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linear(

In [12]:
import nlp
import logging

logging.basicConfig(level=logging.INFO)
rouge = nlp.load_metric("rouge", experiment_id=0)
model.encoder.config.gradient_checkpointing = True

# set decoding params
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.max_length = 256
model.config.min_length = 56
model.config.no_repat_ngram_size = 3
model.early_stopping = True
model.length_penalty = 2.0
model.num_beams = 4

encoder_length = 4096
decoder_length = 128
batch_size = 4

INFO:nlp.load:Checking /home/ratan_singh/.cache/huggingface/datasets/5ecb6e4b474317b41ae1fe5d702d1af8d86d452f0b1d70f77a12f6f014ded6ac.35bc2c477aa456d2f589656477ccb0b463c21cdfb83a9de86d63de8560a96d1b.py for additional imports.
INFO:filelock:Lock 140276852705504 acquired on /home/ratan_singh/.cache/huggingface/datasets/5ecb6e4b474317b41ae1fe5d702d1af8d86d452f0b1d70f77a12f6f014ded6ac.35bc2c477aa456d2f589656477ccb0b463c21cdfb83a9de86d63de8560a96d1b.py.lock
INFO:nlp.load:Found main folder for metric https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/rouge/rouge.py at /home/ratan_singh/anaconda3/lib/python3.8/site-packages/nlp/metrics/rouge
INFO:nlp.load:Found specific version folder for metric https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/rouge/rouge.py at /home/ratan_singh/anaconda3/lib/python3.8/site-packages/nlp/metrics/rouge/06783dbed5f6b6a5413f84d2a5f0d9dc9cb871f1aeb3787f2c90a8e3fe60b1c1
INFO:nlp.load:Found script file from https://s3.amazonaws.com/datasets.hug

In [13]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=encoder_length,
        add_special_tokens=True,
    )
    outputs = tokenizer(
        batch["target"],
        padding="max_length",
        truncation=True,
        max_length=decoder_length,
        add_special_tokens=True,
    )

    
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # set 128 tokens to global attention
    batch["global_attention_mask"] = [[1 if i < 128 else 0 for i in range(sequence_length)] for sequence_length in len(inputs.input_ids) * [encoder_length]]
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]
    batch["decoder_attention_mask"] = outputs.attention_mask
    
    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])


    return batch

In [14]:
train_dataset = train_rank_dataset_tot.select(range(40000))
val_dataset = val_rank_dataset_tot.select(range(3000))

In [15]:
train_dataset[7715]["text"]

"nowadays , post - operative nausea and vomiting ( ponv ) sound to be the leading cause of morbidity and discomfort among pediatric surgical patients .ponv may at least direct patients to have longer stay at hospital after surgery but some more serious consequences such as bleeding , wound dehiscence , dehydration , electrolyte disturbance and/or pulmonary aspiration of gastric contents can be resulted by severe types of ponv[2 , 3 ] .the incidence of ponv is higher in pediatric patients than in adults[2 , 4 ] .it has been reported that up to 40% of children undergoing appendicectomy who have not received any antiemetic drug may have ponv .clonidine , an 2-adrenoceptor agonist , is an effective premedicant which has been shown to decrease the incidence of ponv in children undergoing strabismus surgery[6 , 7 ] , but its efficacy for prevention of ponv after appendectomy in pediatric patients has not been studied yet .we therefore decided to evaluate the effect of oral clonidine given pr

In [16]:
import torch

train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["text", "target"],
)

val_dataset = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["text", "target"],
)

train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

  0%|          | 0/10000 [00:00<?, ?ba/s]

  0%|          | 0/750 [00:00<?, ?ba/s]

In [17]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid
    

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [18]:
from transformers import TrainingArguments
from transformers import Trainer

# set training arguments - these params are not really tuned, feel free to change
training_args = TrainingArguments(
    output_dir="./",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    do_train=True,
    do_eval=True,
    logging_steps=500,
    save_steps=500,
    eval_steps=500,
    eval_accumulation_steps = 100,
    overwrite_output_dir=True,
    warmup_steps=100,
    save_total_limit=5,
    num_train_epochs=1,
)

# instantiate trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [19]:
trainer.train()

Step,Training Loss
500,2.841200
1000,2.539000
1500,2.414300
2000,2.391600
2500,2.317600
3000,2.267200
3500,2.299000
4000,2.204300
4500,2.235100
5000,2.183800


TrainOutput(global_step=10000, training_loss=2.2320327392578125, metrics={'train_runtime': 11883.6749, 'train_samples_per_second': 0.841, 'total_flos': 305875455621120000, 'epoch': 1.0})

In [20]:
!nvidia-smi

Wed Dec  1 20:09:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    48W / 350W |  13135MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
FILE = "model1_40ktrain_ranked_temporal.pth"

torch.save(model.state_dict(), FILE)

In [59]:
test_rank_dataset_tot_pd = pd.read_json('data_david/ranked_temporal_test.json')

In [60]:
for i in range(test_rank_dataset_tot_pd.shape[0]):
    if isinstance(test_rank_dataset_tot_pd['text'].get(i),str) == False:
        test_rank_dataset_tot_pd.drop(i,inplace=True)

In [61]:
test_rank_dataset_tot = Dataset.from_pandas(test_rank_dataset_tot_pd)

In [62]:
import torch

with torch.no_grad():
    #test_dataset = nlp.load_dataset("scientific_papers", "pubmed", split="test")
    test_dataset = test_rank_dataset_tot.select(range(59))
    batch_size = 4

    encoder_length = 4096
    decoder_length = 512

    # map data correctly
    def generate_summary(batch):
        # Tokenizer will automatically set [BOS] <text> [EOS]
        # cut off at BERT max length 512
        inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=encoder_length, return_tensors="pt")
        input_ids = inputs.input_ids.to("cuda")
        attention_mask = inputs.attention_mask.to("cuda")
        global_attention_mask = torch.zeros_like(attention_mask)
        global_attention_mask[:, :decoder_length] = 1

        outputs = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)

        # all special tokens including will be removed
        output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

        batch["pred"] = output_str

        return batch


    results = test_dataset.map(generate_summary, batched=True, batch_size=batch_size)

    # load rouge for validation
    rougen = nlp.load_metric("rouge", experiment_id=5)

    pred_str = results["pred"]
    label_str = results["target"]

    rouge_output = rougen.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    print(rouge_output)

  0%|          | 0/15 [00:00<?, ?ba/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/ratan_singh/anaconda3/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Score(precision=0.16124734977837824, recall=0.07117631821726844, fmeasure=0.09377137524377968)


In [24]:
test_dataset_tot.shape

(6658, 3)

In [46]:
results["pred"][3]

'<S> objective : to evaluate the prevalence of hormone levels in the postnatal period and the risk factors for the development of hyperhyperglycemia ( cah ) in the classical cah. </S><S> the study was conducted to determine the prevalence and risk of cah in the clinical period of the classical and post - adolescent cah period.materials and methods : the study included a cohort of 518 patients with the classic cah of the late adolescent period. </><S><RESULTS : the patients were selected from the classical cervical cah and the biliary centre. </>'

In [47]:
results["abstract"][3]

' congenital adrenal hyperplasia is a group of autosomal recessive disorders caused by enzyme deficiency which leads to defects in biosynthesis of steroid precursors . \n most common is 21 hydroxylase deficiency . \n clinical spectrum varies from non - classical cah to classic cah , and it may be simple virilising form or salt - wastinfg type . \n 29 patients were included in our study from january 2012 to october 2012 . \n 76% were females . \n male babies typically presented with adrenal crisis between 3rd to 6th week of life . around 20% of females \n were identified and appropriately treated only after late adolescence . \n short stature was seen in 1/3rd of patients . \n 1/3rd of patients had suppressed 17 ohp levels suggestive of over - replacement therapy which may contribute to final reduction in adult height . '

LOAD the SAVED MODEL

In [2]:
from datasets import load_dataset, load_metric
from transformers import LongformerTokenizer, EncoderDecoderModel

model2 = EncoderDecoderModel.from_pretrained("patrickvonplaten/longformer2roberta-cnn_dailymail-fp16")
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Some weights of the model checkpoint at patrickvonplaten/longformer2roberta-cnn_dailymail-fp16 were not used when initializing EncoderDecoderModel: ['decoder.roberta.pooler.dense.weight', 'decoder.roberta.pooler.dense.bias']
- This IS expected if you are initializing EncoderDecoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EncoderDecoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
import torch 

FILE = "model1_40ktrain_ranked_temporal.pth"

model2.load_state_dict(torch.load(FILE,map_location='cuda'))

<All keys matched successfully>

In [4]:
model2.eval()
model2.to('cuda')

EncoderDecoderModel(
  (encoder): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0): LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linear(

In [6]:
import pandas as pd

test_rank_dataset_tot_pd = pd.read_json('data_david/ranked_test.json')

In [7]:
for i in range(test_rank_dataset_tot_pd.shape[0]):
    if isinstance(test_rank_dataset_tot_pd['text'].get(i),str) == False:
        test_rank_dataset_tot_pd.drop(i,inplace=True)

In [10]:
test_rank_dataset_tot = Dataset.from_pandas(test_rank_dataset_tot_pd)

In [15]:
test_rank_dataset_tot.shape

(6658, 3)

In [16]:
import torch
import nlp

with torch.no_grad():
    test_dataset_new = test_rank_dataset_tot
    batch_size = 4

    encoder_length = 4096
    decoder_length = 512

    # map data correctly
    def generate_summary(batch):
        # Tokenizer will automatically set [BOS] <text> [EOS]
        # cut off at BERT max length 512
        inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=encoder_length, return_tensors="pt")
        input_ids = inputs.input_ids.to("cuda")
        attention_mask = inputs.attention_mask.to("cuda")
        global_attention_mask = torch.zeros_like(attention_mask)
        global_attention_mask[:, :decoder_length] = 1

        outputs_new = model2.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)

        # all special tokens including will be removed
        output_str_new = tokenizer.batch_decode(outputs_new, skip_special_tokens=True)

        batch["pred_new"] = output_str_new

        return batch


    results_new = test_dataset_new.map(generate_summary, batched=True, batch_size=batch_size)

    # load rouge for validation
    rougen = nlp.load_metric("rouge", experiment_id=5)

    pred_str_new = results_new["pred_new"]
    label_str_new = results_new["target"]

    rouge_output_new = rougen.compute(predictions=pred_str_new, references=label_str_new, rouge_types=["rouge2"])["rouge2"].mid

    print(rouge_output_new)

  0%|          | 0/1665 [00:00<?, ?ba/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Score(precision=0.16623062788815104, recall=0.067784439515863, fmeasure=0.09267072186746278)


In [18]:
results_new.to_csv('summaries_rank_temporal_test_model2.csv')

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

129295389